# Technical note: OpenCL simulation

These technical notes describe the OpenCL simulation implemented in Myokit.
This is designed for rectangular grids (or more complicated meshes) of _coupled cells,_ connected via a _diffusion current_.
It is also possible to disable the diffusion current, and use it to perform single-cell simulations in parallel.

We assume each cell model contains a membrane potential variable, defined by an equation:

\begin{equation}
\frac{d V}{d t}=-\frac{1}{C_{m}}(I_{\mathit{ion}}+I_{\mathit{stim}}+I_{\mathit{diff}})\label{eq:dv/dt}
\end{equation}

where $I_\mathit{diff}$ is a diffusion current

\begin{equation}
I_{\mathit{diff},i}=\sum_{j\in N_{i}}g_{ij}\left(V_{i}-V_{j}\right)\label{eq:idiff}
\end{equation}

and 

- $N_{i}$ is the set of neighbours of cell $i$;
- $g_{ij}$ is the conductance between cell $i$ and its neighbour $j$.

## Specifying conductances

As of version 1.33.0, Myokit provides three ways of specifying the conductances $g_{ij}$.

### Rectangular, homogeneous conductances

In this mode, we assume either a "strand" (in 1d) or a rectangular grid (2d) of cells, connected with a single conductance $g_x$ (1d) or a horizontal and a vertial conductance $(g_x, g_y)$ (2d).

### Rectangular, heterogeneous conductances

In this mode, we still assume a strand or a rectangular grid, but each conductance is set separately.

In the 1d case, for $n$ cells we, this means we need a list of $n - 1$ conductances.

In the 2d case, for $(m \times n)$ cells, we need two matrices. One of $(m - 1 \times n)$ conductances in the direction of the first coordinate, and one of $(m \times n - 1)$ conductances in the direction of the second coordinate.

### Arbitrary geometries

In this mode, the simulation is initialised in 1d mode, so that each cell is identified by a single subscript $i$ or $j$.
Next, the use specifies a list of connections

## Heterogeneous parameters (scalar fields)

The `SimulationOpenCL` allows _literal constants_ (constants that do not depend on other constants) to be set using a *scalar field*, i.e. to specify a value per cell.

## Heterogeneous pacing

In most simulations, only a few cells are paced.
The `SimulationOpenCL` allows the paced cells to be specificied either explicitly (using a list of cell indices) or by specifying a rectangle of paced cells.

All paced cells receive the same pacing signal.
However, it is possible to set up more complex pacing e.g. by defining a stimulus current as $I_i = p_i * u(t)$ and setting $p_i$ to vary from cell to cell.

## Relationship to single-cell notation

As before, we can write the system of ODEs for each cell as:

\begin{equation}
\dot{y}(t)=f\left(y(t),u(t),t, p\right)
\end{equation}

but we now need to take into account that a cell i's external input $u_i$ depends on $y_i$, plus those of its neighbours:

\begin{equation}
\dot{y_i}(t)=f\left(y_i(t),u_i(y, t), t, p_i\right)
\end{equation}

Here we write $f$ for each cell, instead of $f_i$, to indicate the function is the same (we use the same model for every cell).

However, we write $u_i$ to indicate diffusion currents and pacing vary from cell to cell, and $p_i$ to indicate the possible use of scalar fields.

## Relationship to monodomain model

The monodomain model (see [Leon & Horacek, 1991](https://doi.org/10.1016/0022-0736(91)90077-Y) or [Wikipedia](https://en.wikipedia.org/wiki/Monodomain_model )) can be used to simulate the electrophysiology of large 2 and 3d tissues.
Instead of explicitly defining cells and connections between cells, this model assumes that each point in space can be described as having some cell properties, and some conductive properties.

Under the monodomain model, the electrophysiology of each point in space is described by:

\begin{equation}
\chi \left(
c_m \frac{\partial V}{\partial t}
+ i_\text{ion}
+ i_\text{stim}
\right)
=
\frac{k}{k + 1}
\nabla \cdot D_i \nabla V
\end{equation}



where

$V$ is the local membrane potential (e.g. in V),

$i_\text{ion}$ is the transmembrane current per unit area (e.g. in A/m^2),

$i_\text{stim}$ is the applied stimulus current per unit area (e.g. in A/m^2),

$c_m$ is the membrane capacitance per unit area (e.g. in C/m^2, but usually given as $1\mu F/\textit{cm}^2$),

$\chi$ is the surface area of the membrane per unit of volume (e.g. in m^2/m^3),

$D_i$ is a tensor representing intracellular conductivity (e.g. in S/m), and

$k$ is the ratio between intra- and extracellular conductivity.

Note that we have defined $i_\text{stim}$ here to use the same sign convention as the other transmembrane currents, so that a negative current will excite the cell. This is different from many classical discussions, where the sign of this current is reversed.

Rewriting, we find

\begin{align}
\frac{\partial V}{\partial t}
&=
-\frac{1}{c_m}
\left(
      i_\text{ion}
    + i_\text{stim}
    -\frac{1}{\chi}
    \frac{k}{k + 1}
    \nabla \cdot D_i \nabla V
\right) \\
&= -\frac{1}{c_m}(i_\text{ion} + i_\text{stim} + i_\text{diff})
\end{align}

so that

\begin{equation}
i_\text{diff}
    = -\frac{1}{\chi} \frac{k}{k + 1} \nabla \cdot D_i \nabla V
    = -\nabla \cdot \tilde{D} \nabla V
\end{equation}

where 

\begin{equation}
    \tilde{D} = \frac{1}{\chi}\frac{k}{k + 1}D_i
\end{equation}

**Assuming $\tilde{D}$ does not vary spatially**, we can use [finite differences](https://en.wikipedia.org/wiki/Finite_difference#Higher-order_differences) to discretise this on a 1-dimensional regular grid, and obtain an expression for the diffusion current in node $i$

\begin{align}
i_{\text{diff}, i}
    &= -\nabla \cdot \tilde{D} \nabla V \\
    &= -\tilde{D} \frac{\partial^{2}V}{\partial x^{2}} \\
    &= -\tilde{D} \left( \frac{V_{i + 1} - 2 V_i + V_{i - 1}}{\Delta x^{2}}\right) \\
    &=  \tilde{D} \frac{1}{\Delta x^{2}}\sum_{j \in N}\left(V_i-V_j\right)
\end{align}

where $\Delta x$ is the spacing between subsequent points on the grid, and the sum is taken over the set of points neighbouring $i$.
We can use this same equation for points at the edges, which is equivalent to assuming zero-flux boundary conditions.

(Note that the $i$ and $j$ in the equations above denote a cell index, while the $i$ in $D_i$ is for _internal_.)

Using this expression, we can write

\begin{equation}
\frac{dV}{dt} = -\frac{1}{c_m} (i_\text{ion} + i_\text{stim} + i_\text{diff})
\end{equation}

For models with currents in $\mu A / \textit{cm}^2$ we can use this equation directly.
To use unnormalised currents, we replace the specific capacitance with total capacitance, to find

\begin{equation}
\frac{dV}{dt} = -\frac{1}{C_m} (I_\text{ion} + I_\text{stim} + I_\text{diff})
\end{equation}

### Two-dimensions

In two dimensions, and assuming a diagonal conductivity tensor with values $\tilde{D_x}$ and $\tilde{D_y}$, we can follow a similar procedure on a grid with spacing $\Delta x$ and $\Delta y$ to find

\begin{align}
i_\text{diff}
    = \frac{\tilde{D_x}}{\Delta x^{2}} \sum_{j \in N_x}(V_i - V_j)
    + \frac{\tilde{D_y}}{\Delta y^{2}} \sum_{j \in N_y}(V_i - V_j) \\
\end{align}

where $N_x$ and $N_y$ are the sets of neighbours in the directions corresponding to $\tilde{D_x}$ and $\tilde{D_y}$ respectively.

### Converting parameters

Assuming a rectangular grid with spacing $\Delta x$ in all directions and a spatially homogeneous conductance $\tilde{D}$ or $D_{i}$, we can write

\begin{equation}
i_\text{diff}
    = \tilde{D} \frac{1}{\Delta x^{2}}\sum_{j \in N}\left(V_{i}-V_{x}\right)
    = \bar{g} \sum_{j \in N}\left(V_{i}-V_{x}\right)
\end{equation}

to find

\begin{equation}
\bar{g} = \frac{\tilde{D}}{\Delta x^2} = \frac{1}{\Delta x^2}\frac{1}{\chi}\frac{k}{k + 1}D_i
\end{equation}

where $\bar{g}$ is cell-to-cell conductance per unit membrane area.

In unnormalised form, we obtain

\begin{equation}
g = A_m \frac{1}{\Delta x^2}\frac{1}{\chi}\frac{k}{k + 1}D_i
\end{equation}

where $A_m$ is the surface area of the cell membrane encompassed by each "node" of our discretisation.

### Converting parameters: heterogeneity

Please note, however, that $g_{ij}$ is defined *between* cells/nodes while $\tilde{D}$ is defined *for* each node.
This difference is unproblematic in spatially homogeneous simulations, but leads to some difficulties if hetereogeneity is introduced.

For example, if the conductivity can vary spatially, then $\nabla \cdot \tilde{D} \nabla V \neq \tilde{D} \frac{\partial^{2}V}{\partial x^{2}} $, and the equation for $g_{ij}$ becomes dependent on the spatial derivative of $\tilde{D}$.

Similarly, when using unnormalised models there is no way to choose an $A_m$ *per connection*, rather than per cell, unless we assume $A_m$ does not vary spatially.

## Solver

The solver used in the `SimulationOpenCL` class is a simple forward-Euler method (although Rush-Larsen updates can be used for Hodgkin-Huxley gating equations, if desired).
This has the advantage of being quite a _stable_ method, meaning it can take relatively large steps (larger than methods that take _slightly_ more complex explicit update steps, but not as large as _implicit_ methods).
It also has a low computational cost _per step_, requiring only a single evaluation of the RHS.
A major downside is the requirement to choose a good step size.

There are some plans to build an adaptive method in Myokit, still based on forward Euler.
However, it is worth noting that, as [Garcia-Molla et al.](https://doi.org/10.1016/j.compbiomed.2013.10.023) point out, multi-cell simulations are often of re-entrant arrhythmias in which there are always depolarising cells which require the smallest step size.
In their work, they found fixed step-size methods often outperformed adaptive ones.